In [1]:
import json
import os
import time

import get_youtube8m_features as yt_features
import frame_level_models
import video_level_models
import readers
import utils
import eval_util
import export_model
import losses

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow import app
from tensorflow import gfile
from tensorflow import logging
from tensorflow.python.client import device_lib

flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
location = "jungle" # "urban

In [3]:
train_dir = "models/models_" + location
train_data_pattern = "tfrecords/train_" + location + "_new/*.tfrecord"
num_classes = 16

if(location=="jungle"):
    num_classes = 14

In [4]:
## Maak alle flags aan

# Dataset flags
flags.DEFINE_string("feature_names", "audio_embedding", "Name of the feature "
                  "to use for training.")
flags.DEFINE_string("feature_sizes", "128", "Length of the feature vectors.")

In [5]:
# Model flags
flags.DEFINE_bool(
  "frame_features", True,
  "If set, then --train_data_pattern must be frame-level features. "
  "Otherwise, --train_data_pattern must be aggregated video-level "
  "features. The model must also be set appropriately (i.e. to read 3D "
  "batches VS 4D batches.")
flags.DEFINE_string(
  "model", "FrameLevelLogisticModel",
  "Which architecture to use for the model. Models are defined "
  "in models.py.")
flags.DEFINE_bool(
  "start_new_model", True,
  "If set, this will not resume from a checkpoint and will instead create a"
  " new model instance.")

In [6]:
# Training flags
flags.DEFINE_integer("batch_size", 2048,
                   "How many examples to process per batch for training.")
flags.DEFINE_string("label_loss", "CrossEntropyLoss",
                  "Which loss function to use for training the model.")
flags.DEFINE_float("regularization_penalty", 1.0,
                  "How much weight to give to the regularization loss (the label loss has "
                  "a weight of 1).")
flags.DEFINE_float("base_learning_rate", 0.01,
                 "Which learning rate to start with.")
flags.DEFINE_float("learning_rate_decay", 0.95,
                 "Learning rate decay factor to be applied every "
                 "learning_rate_decay_examples.")
flags.DEFINE_float("learning_rate_decay_examples", 4000000,
                 "Multiply current learning rate by learning_rate_decay "
                 "every learning_rate_decay_examples.")
flags.DEFINE_integer("num_epochs", 1000,
                   "How many passes to make over the dataset before "
                   "halting training.")
flags.DEFINE_integer("max_steps", None,
                   "The maximum number of iterations of the training loop.")
flags.DEFINE_integer("export_model_steps", 100,
                   "The period, in number of steps, with which the model "
                   "is exported for batch prediction.")

In [7]:
# Other flags
flags.DEFINE_integer("num_readers", 16,
                   "How many threads to use for reading input files.")
flags.DEFINE_string("optimizer", "AdamOptimizer",
                  "What optimizer class to use.")
flags.DEFINE_float("clip_gradient_norm", 1.0, "Norm to clip gradients to.")

In [8]:
# Load the environment.
env = json.loads(os.environ.get("TF_CONFIG", "{}"))

os.environ["TF_MIN_GPU_MULTIPROCESSOR_COUNT"]="3"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [9]:
# Load the cluster data from the environment.
cluster_data = env.get("cluster", None)
cluster = tf.train.ClusterSpec(cluster_data) if cluster_data else None

In [10]:
# Load the task data from the environment.
task_data = env.get("task", None) or {"type": "master", "index": 0}
task = type("TaskSpec", (object,), task_data)

In [11]:
def task_as_string(task):
  return "/job:%s/task:%s" % (task.type, task.index)

# Logging the version.
logging.set_verbosity(tf.logging.INFO)
logging.info("%s: Tensorflow version: %s.",
           task_as_string(task), tf.__version__)

INFO:tensorflow:/job:master/task:0: Tensorflow version: 1.8.0.


In [12]:
## Functie die elke keer wordt aangeroepen
def find_class_by_name(name, modules):
  """Searches the provided modules for the named class and returns it."""
  modules = [getattr(module, name, None) for module in modules]
  return next(a for a in modules if a)

In [13]:
## Define model
model = find_class_by_name(FLAGS.model,
    [frame_level_models, video_level_models])()

In [14]:
# Convert feature_names and feature_sizes to lists of values.
feature_names, feature_sizes = utils.GetListOfFeatureNamesAndSizes(
    FLAGS.feature_names, FLAGS.feature_sizes)

# Create reader
if FLAGS.frame_features:
    reader = readers.YT8MFrameFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)
else:
    reader = readers.YT8MAggregatedFeatureReader(
        num_classes=num_classes,
        feature_names=feature_names, feature_sizes=feature_sizes)

In [15]:
model_exporter = export_model.ModelExporter(
    frame_features=FLAGS.frame_features,
    model=model,
    reader=reader)

In [16]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
config.gpu_options.allow_growth = True
is_master = (task.type == "master" and task.index == 0)
last_model_export_step = 0
max_steps_reached = False

In [17]:
# Remove old model, if starting new model
if is_master and FLAGS.start_new_model:
    try:
      logging.info(
          "%s: Removing existing train directory.",
          task_as_string(task))
      gfile.DeleteRecursively(train_dir)
    except:
      logging.error(
          "%s: Failed to delete directory " + train_dir +
          " when starting a new model. Please delete it manually and" +
          " try again.", task_as_string(task))

INFO:tensorflow:/job:master/task:0: Removing existing train directory.


In [18]:
"""Starts a server if the execution is distributed."""
if cluster:
    logging.info("%s: Starting trainer within cluster %s.",
               task_as_string(task), cluster.as_dict())
    server = start_server(cluster, task)
    target = server.target
    device_fn = tf.train.replica_device_setter(
        ps_device="/job:ps",
        worker_device="/job:%s/task:%d" % (task.type, task.index),
        cluster=cluster)
else:
    target = ""
    device_fn = ""

In [19]:
def get_meta_filename(start_new_model, train_dir, task):
    if start_new_model:
      logging.info("%s: Flag 'start_new_model' is set. Building a new model.",
                   task_as_string(task))
      return None

    latest_checkpoint = tf.train.latest_checkpoint(train_dir)
    if not latest_checkpoint:
      logging.info("%s: No checkpoint file found. Building a new model.",
                   task_as_string(task))
      return None

    meta_filename = latest_checkpoint + ".meta"
    if not gfile.Exists(meta_filename):
      logging.info("%s: No meta graph file found. Building a new model.",
                     task_as_string(task))
      return None
    else:
      return meta_filename

meta_filename = get_meta_filename(FLAGS.start_new_model, train_dir, task)

INFO:tensorflow:/job:master/task:0: Flag 'start_new_model' is set. Building a new model.


In [20]:
## Functie die wordt aangeroepen door build_graph

def get_input_data_tensors(reader,
                           data_pattern,
                           batch_size=1000,
                           num_epochs=None,
                           num_readers=1):
  """Creates the section of the graph which reads the training data.

  Args:
    reader: A class which parses the training data.
    data_pattern: A 'glob' style path to the data files.
    batch_size: How many examples to process at a time.
    num_epochs: How many passes to make over the training data. Set to 'None'
                to run indefinitely.
    num_readers: How many I/O threads to use.

  Returns:
    A tuple containing the features tensor, labels tensor, and optionally a
    tensor containing the number of frames per video. The exact dimensions
    depend on the reader being used.

  Raises:
    IOError: If no files matching the given pattern were found.
  """
  logging.info("Using batch size of " + str(batch_size) + " for training.")
  with tf.name_scope("train_input"):
    files = gfile.Glob(data_pattern)
    if not files:
      raise IOError("Unable to find training files. data_pattern='" +
                    data_pattern + "'.")
    logging.info("Number of training files: %s.", str(len(files)))
    filename_queue = tf.train.string_input_producer(
        files, num_epochs=num_epochs, shuffle=True)
    training_data = [
        reader.prepare_reader(filename_queue) for _ in range(num_readers)
    ]

    print(training_data)
    
    ## Create batches by randomly shuffling tensors
    return tf.train.shuffle_batch_join(
        training_data,
        batch_size=batch_size,
        capacity=batch_size * 5,
        min_after_dequeue=batch_size,
        allow_smaller_final_batch=True,
        enqueue_many=True)

In [21]:
## Deze functie wordt aangeroepen om de Tensorflow-graph te bouwen

def build_graph(reader,
                model,
                train_data_pattern,
                label_loss_fn=losses.CrossEntropyLoss(),
                batch_size=1000,
                base_learning_rate=0.01,
                learning_rate_decay_examples=1000000,
                learning_rate_decay=0.95,
                optimizer_class=tf.train.AdamOptimizer,
                clip_gradient_norm=1.0,
                regularization_penalty=1,
                num_readers=1,
                num_epochs=None):
  """Creates the Tensorflow graph.

  This will only be called once in the life of
  a training model, because after the graph is created the model will be
  restored from a meta graph file rather than being recreated.

  Args:
    reader: The data file reader. It should inherit from BaseReader.
    model: The core model (e.g. logistic or neural net). It should inherit
           from BaseModel.
    train_data_pattern: glob path to the training data files.
    label_loss_fn: What kind of loss to apply to the model. It should inherit
                from BaseLoss.
    batch_size: How many examples to process at a time.
    base_learning_rate: What learning rate to initialize the optimizer with.
    optimizer_class: Which optimization algorithm to use.
    clip_gradient_norm: Magnitude of the gradient to clip to.
    regularization_penalty: How much weight to give the regularization loss
                            compared to the label loss.
    num_readers: How many threads to use for I/O operations.
    num_epochs: How many passes to make over the data. 'None' means an
                unlimited number of passes.
  """

  global_step = tf.Variable(0, trainable=False, name="global_step")
  print(global_step)
  local_device_protos = device_lib.list_local_devices()
  gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
  num_gpus = len(gpus)
  logging.info("Number of GPUs: %s.", str(num_gpus))

  if num_gpus > 0:
    logging.info("Using the following GPUs to train: " + str(gpus))
    num_towers = num_gpus
    device_string = '/gpu:%d'
  else:
    logging.info("No GPUs found. Training on CPU.")
    num_towers = 1
    device_string = '/cpu:%d'

  learning_rate = tf.train.exponential_decay(
      base_learning_rate,
      global_step * batch_size * num_towers,
      learning_rate_decay_examples,
      learning_rate_decay,
      staircase=True)
  tf.summary.scalar('learning_rate', learning_rate)

  optimizer = optimizer_class(learning_rate)

  ## Read input files
  unused_video_id, model_input_raw, labels_batch, num_frames = (
      get_input_data_tensors(
          reader,
          train_data_pattern,
          batch_size=batch_size * num_towers,
          num_readers=num_readers,
          num_epochs=num_epochs))
  tf.summary.histogram("model/input_raw", model_input_raw)
 
  feature_dim = len(model_input_raw.get_shape()) - 1

  model_input = tf.nn.l2_normalize(model_input_raw, feature_dim)

  tower_inputs = tf.split(model_input, num_towers)
  tower_labels = tf.split(labels_batch, num_towers)
  tower_num_frames = tf.split(num_frames, num_towers)
  tower_gradients = []
  tower_predictions = []
  tower_label_losses = []
  tower_reg_losses = []
  for i in range(num_towers):
    # For some reason these 'with' statements can't be combined onto the same
    # line. They have to be nested.
    with tf.device(device_string % i):
      with (tf.variable_scope(("tower"), reuse=True if i > 0 else None)):
        with (slim.arg_scope([slim.model_variable, slim.variable], device="/cpu:0" if num_gpus!=1 else "/gpu:0")):
          result = model.create_model(
            tower_inputs[i],
            num_frames=tower_num_frames[i],
            vocab_size=reader.num_classes,
            labels=tower_labels[i])
          #print(tower_labels[i])
          for variable in slim.get_model_variables():
            tf.summary.histogram(variable.op.name, variable)

          predictions = result["predictions"]
          tower_predictions.append(predictions)

          if "loss" in result.keys():
            label_loss = result["loss"]
          else:
            label_loss = label_loss_fn.calculate_loss(predictions, tower_labels[i])

          if "regularization_loss" in result.keys():
            reg_loss = result["regularization_loss"]
          else:
            reg_loss = tf.constant(0.0)

          reg_losses = tf.losses.get_regularization_losses()
          if reg_losses:
            reg_loss += tf.add_n(reg_losses)

          tower_reg_losses.append(reg_loss)

          # Adds update_ops (e.g., moving average updates in batch normalization) as
          # a dependency to the train_op.
          update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
          if "update_ops" in result.keys():
            update_ops += result["update_ops"]
          if update_ops:
            with tf.control_dependencies(update_ops):
              barrier = tf.no_op(name="gradient_barrier")
              with tf.control_dependencies([barrier]):
                label_loss = tf.identity(label_loss)

          tower_label_losses.append(label_loss)

          # Incorporate the L2 weight penalties etc.
          final_loss = regularization_penalty * reg_loss + label_loss
          gradients = optimizer.compute_gradients(final_loss,
              colocate_gradients_with_ops=False)
          tower_gradients.append(gradients)
  label_loss = tf.reduce_mean(tf.stack(tower_label_losses))
  tf.summary.scalar("label_loss", label_loss)
  if regularization_penalty != 0:
    reg_loss = tf.reduce_mean(tf.stack(tower_reg_losses))
    tf.summary.scalar("reg_loss", reg_loss)
  merged_gradients = utils.combine_gradients(tower_gradients)

  if clip_gradient_norm > 0:
    with tf.name_scope('clip_grads'):
      merged_gradients = utils.clip_gradient_norms(merged_gradients, clip_gradient_norm)

  train_op = optimizer.apply_gradients(merged_gradients, global_step=global_step)

  tf.add_to_collection("global_step", global_step)
  tf.add_to_collection("loss", label_loss)
  tf.add_to_collection("predictions", tf.concat(tower_predictions, 0))
  tf.add_to_collection("input_batch_raw", model_input_raw)
  tf.add_to_collection("input_batch", model_input)
  tf.add_to_collection("num_frames", num_frames)
  tf.add_to_collection("labels", tf.cast(labels_batch, tf.float32))
  tf.add_to_collection("train_op", train_op)

In [22]:
with tf.Graph().as_default() as graph:
    
    if meta_filename:
        logging.info("%s: Restoring from meta graph file %s",
            task_as_string(task), meta_filename)
        saver = tf.train.import_meta_graph(meta_filename)
    
    with tf.device(device_fn):

        if not meta_filename:
            """Find the model and build the graph."""

            label_loss_fn = find_class_by_name(FLAGS.label_loss, [losses])()
            optimizer_class = find_class_by_name(FLAGS.optimizer, [tf.train])

            build_graph(reader=reader,
                         model=model,
                         optimizer_class=optimizer_class,
                         clip_gradient_norm=FLAGS.clip_gradient_norm,
                         train_data_pattern=train_data_pattern,
                         label_loss_fn=label_loss_fn,
                         base_learning_rate=FLAGS.base_learning_rate,
                         learning_rate_decay=FLAGS.learning_rate_decay,
                         learning_rate_decay_examples=FLAGS.learning_rate_decay_examples,
                         regularization_penalty=FLAGS.regularization_penalty,
                         num_readers=FLAGS.num_readers,
                         batch_size=FLAGS.batch_size,
                         num_epochs=FLAGS.num_epochs)

        saver = tf.train.Saver(max_to_keep=0, keep_checkpoint_every_n_hours=0.25)
    
        global_step = tf.get_collection("global_step")[0]
        loss = tf.get_collection("loss")[0]
        predictions = tf.get_collection("predictions")[0]
        labels = tf.get_collection("labels")[0]
        train_op = tf.get_collection("train_op")[0]
        init_op = tf.global_variables_initializer()
        
    sv = tf.train.Supervisor(
        graph,
        logdir=train_dir,
        init_op=init_op,
        is_chief=is_master,
        global_step=global_step,
        save_model_secs=15 * 60,
        save_summaries_secs=120,
        saver=saver)

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>
INFO:tensorflow:Number of GPUs: 1.
INFO:tensorflow:Using the following GPUs to train: ['/device:GPU:0']
INFO:tensorflow:Using batch size of 2048 for training.
INFO:tensorflow:Number of training files: 5116.
[(<tf.Tensor 'train_input/ExpandDims:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_1:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_2:0' shape=(1, 14) dtype=bool>, <tf.Tensor 'train_input/ExpandDims_3:0' shape=(1,) dtype=int32>), (<tf.Tensor 'train_input/ExpandDims_4:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_5:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_6:0' shape=(1, 14) dtype=bool>, <tf.Tensor 'train_input/ExpandDims_7:0' shape=(1,) dtype=int32>), (<tf.Tensor 'train_input/ExpandDims_8:0' shape=(1,) dtype=string>, <tf.Tensor 'train_input/ExpandDims_9:0' shape=(1, 300, 128) dtype=float32>, <tf.Tensor 'train_input/ExpandDims_10:0' sha

In [23]:
def export_model(global_step_val, saver, save_path, session):

    # If the model has already been exported at this step, return.
    if global_step_val == last_model_export_step:
      return

    last_checkpoint = saver.save(session, save_path, global_step_val)

    model_dir = "{0}/export/step_{1}".format(train_dir, global_step_val)
    logging.info("%s: Exporting the model at step %s to %s.",
                 task_as_string(task), global_step_val, model_dir)

    model_exporter.export_model(
        model_dir=model_dir,
        global_step_val=global_step_val,
        last_checkpoint=last_checkpoint)

In [24]:
logging.info("%s: Starting managed session.", task_as_string(task))

with sv.managed_session(target, config=config) as sess:
    
    start = time.time()
    
    try:
        logging.info("%s: Entering training loop.", task_as_string(task))
        while (not sv.should_stop()) and (not max_steps_reached):
            batch_start_time = time.time()
            
            _, global_step_val, loss_val, predictions_val, labels_val = sess.run(
                [train_op, global_step, loss, predictions, labels])
            
            seconds_per_batch = time.time() - batch_start_time
            examples_per_second = labels_val.shape[0] / seconds_per_batch
            
            if FLAGS.max_steps and FLAGS.max_steps <= global_step_val:
                max_steps_reached = True

            if is_master and global_step_val % 10 == 0 and train_dir:
                eval_start_time = time.time()
                hit_at_one = eval_util.calculate_hit_at_one(predictions_val, labels_val)
                perr = eval_util.calculate_precision_at_equal_recall_rate(predictions_val,
                                                                  labels_val)
                gap = eval_util.calculate_gap(predictions_val, labels_val)
                eval_end_time = time.time()
                eval_time = eval_end_time - eval_start_time

                logging.info("training step " + str(global_step_val) + " | Loss: " + ("%.2f" % loss_val) +
                             " Examples/sec: " + ("%.2f" % examples_per_second) + " | Hit@1: " +
                            ("%.2f" % hit_at_one) + " PERR: " + ("%.2f" % perr) +
                             " GAP: " + ("%.2f" % gap))

                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Hit@1", hit_at_one),
                    global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_Perr", perr), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("model/Training_GAP", gap), global_step_val)
                sv.summary_writer.add_summary(
                    utils.MakeSummary("global_step/Examples/Second",
                                      examples_per_second), global_step_val)
                sv.summary_writer.flush()

                # Exporting the model every x steps
                time_to_export = ((last_model_export_step == 0) or
                    (global_step_val - last_model_export_step
                     >= FLAGS.export_model_steps))

                if is_master and time_to_export:
                    export_model(global_step_val, sv.saver, sv.save_path, sess)
                    last_model_export_step = global_step_val
            else:
                logging.info("training step " + str(global_step_val) + " | Loss: " +
                            ("%.2f" % loss_val) + " Examples/sec: " + ("%.2f" % examples_per_second))
    except tf.errors.OutOfRangeError:
        logging.info("%s: Done training -- epoch limit reached")

logging.info("%s: Exited training loop.", task_as_string(task))
sv.Stop()

end = time.time()
print("Time elapsed: " + str(end - start))

INFO:tensorflow:/job:master/task:0: Starting managed session.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path models/models_jungle\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:/job:master/task:0: Entering training loop.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:models/models_jungle\model.ckpt-0 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:training step 1 | Loss: 9.72 Examples/sec: 777.30
INFO:tensorflow:training step 2 | Loss: 9.59 Examples/sec: 2302.29
INFO:tensorflow:training step 3 | Loss: 9.48 Examples/sec: 2797.81
INFO:tensorflow:Recording summary at step 3.
INFO:tensorflow:training step 4 | Loss: 9.37 Examples/sec: 2649.65
INFO:tensorflow:training step 5 | Loss: 9.27 Examples/sec: 2772.42
INFO:tensorflow:training step 6 | Loss: 9.15 Examples/sec: 2716.24
INFO:tensorflow:training step 7 | Loss: 9.02 Exa

INFO:tensorflow:training step 102 | Loss: 3.68 Examples/sec: 2459.26
INFO:tensorflow:training step 103 | Loss: 3.71 Examples/sec: 2449.15
INFO:tensorflow:training step 104 | Loss: 3.65 Examples/sec: 2447.54
INFO:tensorflow:training step 105 | Loss: 3.64 Examples/sec: 2438.81
INFO:tensorflow:training step 106 | Loss: 3.60 Examples/sec: 2441.71
INFO:tensorflow:training step 107 | Loss: 3.59 Examples/sec: 2450.45
INFO:tensorflow:training step 108 | Loss: 3.65 Examples/sec: 2333.50
INFO:tensorflow:training step 109 | Loss: 3.66 Examples/sec: 2315.09
INFO:tensorflow:training step 110 | Loss: 3.64 Examples/sec: 2251.62 | Hit@1: 0.66 PERR: 0.66 GAP: 0.62
INFO:tensorflow:models/models_jungle\model.ckpt-110 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 110 to models/models_jungle/export/step_110.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-110
INFO:tensorflow:No assets to save.
INFO:tensorf

INFO:tensorflow:training step 208 | Loss: 2.50 Examples/sec: 2274.06
INFO:tensorflow:training step 209 | Loss: 2.49 Examples/sec: 2298.21
INFO:tensorflow:training step 210 | Loss: 2.55 Examples/sec: 2341.48 | Hit@1: 0.70 PERR: 0.70 GAP: 0.70
INFO:tensorflow:models/models_jungle\model.ckpt-210 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/job:master/task:0: Exporting the model at step 210 to models/models_jungle/export/step_210.
INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-210
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_210\\saved_model.pb'
INFO:tensorflow:training step 211 | Loss: 2.50 Examples/sec: 4984.18
INFO:tensorflow:training step 212 | Loss: 2.49 Examples/sec: 2404.54
INFO:tensorflow:training step 213 | Loss: 2.52 Examples/sec: 2322.94
INFO:tensorflow:training step 214 | Loss: 2.47 Examples/sec: 2312.48
INFO:tensorflow:tra

INFO:tensorflow:Restoring parameters from models/models_jungle\model.ckpt-310
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'models/models_jungle/export/step_310\\saved_model.pb'
INFO:tensorflow:training step 311 | Loss: 2.18 Examples/sec: 4786.66
INFO:tensorflow:training step 312 | Loss: 2.19 Examples/sec: 2354.90
INFO:tensorflow:training step 313 | Loss: 2.03 Examples/sec: 2196.23
INFO:tensorflow:training step 314 | Loss: 2.01 Examples/sec: 2184.55
INFO:tensorflow:training step 315 | Loss: 2.03 Examples/sec: 2224.23
INFO:tensorflow:training step 316 | Loss: 1.98 Examples/sec: 2212.80
INFO:tensorflow:training step 317 | Loss: 2.08 Examples/sec: 2215.19
INFO:tensorflow:training step 318 | Loss: 2.07 Examples/sec: 2196.24
INFO:tensorflow:training step 319 | Loss: 2.00 Examples/sec: 2198.59
INFO:tensorflow:training step 320 | Loss: 1.96 Examples/sec: 2193.89 | Hit@1: 0.76 PERR: 0.75 GAP: 0.79
INFO:tensorflow:training step 3

INFO:tensorflow:training step 414 | Loss: 1.95 Examples/sec: 2015.19
INFO:tensorflow:training step 415 | Loss: 1.89 Examples/sec: 2005.35
INFO:tensorflow:training step 416 | Loss: 1.87 Examples/sec: 1998.59
INFO:tensorflow:training step 417 | Loss: 1.73 Examples/sec: 1995.61
INFO:tensorflow:training step 418 | Loss: 1.80 Examples/sec: 1979.47
INFO:tensorflow:training step 419 | Loss: 1.83 Examples/sec: 2081.59
INFO:tensorflow:training step 420 | Loss: 1.85 Examples/sec: 1968.82 | Hit@1: 0.74 PERR: 0.73 GAP: 0.77
INFO:tensorflow:training step 421 | Loss: 1.87 Examples/sec: 2790.06
INFO:tensorflow:training step 422 | Loss: 1.84 Examples/sec: 1952.67
INFO:tensorflow:training step 423 | Loss: 1.76 Examples/sec: 1976.40
INFO:tensorflow:training step 424 | Loss: 1.75 Examples/sec: 1980.15
INFO:tensorflow:training step 425 | Loss: 1.77 Examples/sec: 1982.29
INFO:tensorflow:training step 426 | Loss: 1.77 Examples/sec: 1976.40
INFO:tensorflow:training step 427 | Loss: 1.77 Examples/sec: 1951.98

...

INFO:tensorflow:training step 8041 | Loss: 1.17 Examples/sec: 2014.05
INFO:tensorflow:training step 8042 | Loss: 1.20 Examples/sec: 1702.97
INFO:tensorflow:training step 8043 | Loss: 1.30 Examples/sec: 1692.67
INFO:tensorflow:training step 8044 | Loss: 1.18 Examples/sec: 1692.69
INFO:tensorflow:training step 8045 | Loss: 1.15 Examples/sec: 1699.92
INFO:tensorflow:training step 8046 | Loss: 1.21 Examples/sec: 1677.12
INFO:tensorflow:training step 8047 | Loss: 1.18 Examples/sec: 1701.40
INFO:tensorflow:training step 8048 | Loss: 1.16 Examples/sec: 1691.59
INFO:tensorflow:training step 8049 | Loss: 1.23 Examples/sec: 1695.85
INFO:tensorflow:training step 8050 | Loss: 1.21 Examples/sec: 1693.36 | Hit@1: 0.79 PERR: 0.78 GAP: 0.84
INFO:tensorflow:training step 8051 | Loss: 1.28 Examples/sec: 2086.10
INFO:tensorflow:training step 8052 | Loss: 1.23 Examples/sec: 1722.79
INFO:tensorflow:training step 8053 | Loss: 1.23 Examples/sec: 1741.67
INFO:tensorflow:training step 8054 | Loss: 1.14 Example

INFO:tensorflow:training step 8146 | Loss: 1.15 Examples/sec: 2033.91
INFO:tensorflow:training step 8147 | Loss: 1.26 Examples/sec: 2023.86
INFO:tensorflow:training step 8148 | Loss: 1.25 Examples/sec: 2022.43
INFO:tensorflow:training step 8149 | Loss: 1.27 Examples/sec: 2032.40
INFO:tensorflow:training step 8150 | Loss: 1.26 Examples/sec: 2042.48 | Hit@1: 0.77 PERR: 0.76 GAP: 0.83
INFO:tensorflow:training step 8151 | Loss: 1.22 Examples/sec: 2316.43
INFO:tensorflow:training step 8152 | Loss: 1.23 Examples/sec: 2045.49
INFO:tensorflow:training step 8153 | Loss: 1.15 Examples/sec: 2035.13
INFO:tensorflow:training step 8154 | Loss: 1.19 Examples/sec: 2032.36
INFO:tensorflow:training step 8155 | Loss: 1.22 Examples/sec: 2032.63
INFO:tensorflow:training step 8156 | Loss: 1.16 Examples/sec: 2019.41
INFO:tensorflow:training step 8157 | Loss: 1.15 Examples/sec: 1986.61
INFO:tensorflow:training step 8158 | Loss: 1.17 Examples/sec: 2009.67
INFO:tensorflow:training step 8159 | Loss: 1.19 Example

INFO:tensorflow:training step 8251 | Loss: 1.23 Examples/sec: 2208.16
INFO:tensorflow:training step 8252 | Loss: 1.18 Examples/sec: 1810.04
INFO:tensorflow:training step 8253 | Loss: 1.21 Examples/sec: 1784.38
INFO:tensorflow:training step 8254 | Loss: 1.22 Examples/sec: 1746.76
INFO:tensorflow:training step 8255 | Loss: 1.16 Examples/sec: 1774.72
INFO:tensorflow:training step 8256 | Loss: 1.23 Examples/sec: 1771.71
INFO:tensorflow:training step 8257 | Loss: 1.18 Examples/sec: 1791.36
INFO:tensorflow:training step 8258 | Loss: 1.12 Examples/sec: 1777.14
INFO:tensorflow:training step 8259 | Loss: 1.17 Examples/sec: 1752.58
INFO:tensorflow:training step 8260 | Loss: 1.21 Examples/sec: 1724.51 | Hit@1: 0.79 PERR: 0.78 GAP: 0.84
INFO:tensorflow:training step 8261 | Loss: 1.28 Examples/sec: 2292.93
INFO:tensorflow:training step 8262 | Loss: 1.27 Examples/sec: 1763.87
INFO:tensorflow:training step 8263 | Loss: 1.21 Examples/sec: 1751.26
INFO:tensorflow:training step 8264 | Loss: 1.22 Example

INFO:tensorflow:training step 8356 | Loss: 1.24 Examples/sec: 1770.73
INFO:tensorflow:training step 8357 | Loss: 1.19 Examples/sec: 1795.30
INFO:tensorflow:training step 8358 | Loss: 1.16 Examples/sec: 1788.06
INFO:tensorflow:training step 8359 | Loss: 1.19 Examples/sec: 1787.94
INFO:tensorflow:training step 8360 | Loss: 1.20 Examples/sec: 1810.67 | Hit@1: 0.79 PERR: 0.79 GAP: 0.84
INFO:tensorflow:training step 8361 | Loss: 1.28 Examples/sec: 2168.14
INFO:tensorflow:training step 8362 | Loss: 1.20 Examples/sec: 1812.00
INFO:tensorflow:training step 8363 | Loss: 1.11 Examples/sec: 1802.51
INFO:tensorflow:training step 8364 | Loss: 1.12 Examples/sec: 1806.51
INFO:tensorflow:training step 8365 | Loss: 1.23 Examples/sec: 1816.29
INFO:tensorflow:training step 8366 | Loss: 1.18 Examples/sec: 1805.20
INFO:tensorflow:training step 8367 | Loss: 1.24 Examples/sec: 1835.18
INFO:tensorflow:training step 8368 | Loss: 1.20 Examples/sec: 1833.70
INFO:tensorflow:training step 8369 | Loss: 1.14 Example

INFO:tensorflow:training step 8461 | Loss: 1.24 Examples/sec: 2499.41
INFO:tensorflow:training step 8462 | Loss: 1.30 Examples/sec: 2048.31
INFO:tensorflow:training step 8463 | Loss: 1.19 Examples/sec: 2017.83
INFO:tensorflow:training step 8464 | Loss: 1.23 Examples/sec: 2010.50
INFO:tensorflow:training step 8465 | Loss: 1.19 Examples/sec: 2028.53
INFO:tensorflow:training step 8466 | Loss: 1.17 Examples/sec: 2025.98
INFO:tensorflow:training step 8467 | Loss: 1.12 Examples/sec: 2024.35
INFO:tensorflow:training step 8468 | Loss: 1.19 Examples/sec: 2024.16
INFO:tensorflow:training step 8469 | Loss: 1.25 Examples/sec: 2012.25
INFO:tensorflow:training step 8470 | Loss: 1.21 Examples/sec: 2018.31 | Hit@1: 0.79 PERR: 0.79 GAP: 0.85
INFO:tensorflow:training step 8471 | Loss: 1.20 Examples/sec: 2322.14
INFO:tensorflow:training step 8472 | Loss: 1.19 Examples/sec: 2037.96
INFO:tensorflow:training step 8473 | Loss: 1.18 Examples/sec: 2003.99
INFO:tensorflow:training step 8474 | Loss: 1.16 Example

In [25]:
9629/3600

2.674722222222222